# Lab06 - Intrusion Detection

## General Information

**Assistents**
- Jamila Alsayed Kassem
- Leonardo Boldrini
- Wouter Petri
- Frederick Kreuk
- Kyrian Maat
- Robin Slot
- Emiel Wiedijk
- Robert van Straten
- Ralph Erkamps

**Lab dates**  
13 or 14 October and 18 or 19 October 2022

**Deadline**  
20 October 2022 at 12:00 CEST

**Total points**  
20

## Lab Description

In this lab you will receive several PCAP files. These files will contain network-based attacks. Your task in this assignment is to write code to detect these attacks and classify the type thereof. The attacks are all different types of DDoS attacks. Your perspective is that of the network engineer, trying to protect their network against these DDoS attacks. You want to classify attacks so that you can create filters to omit malevolent traffic.

For the submission of this lab, **you will only have to hand in this Jupyter Notebook**, which should contain all your code and explanations as indicated in the Notebook. Please fill in any missing sections of code, and provide explanations where these are asked.

**This lab must be done in pairs. Before you submit, you have to register into a group on CodeGrade.**

## Setup

The following few blocks contain general imports and functions that you may use during this lab. Of note is the library ScaPy, which provides a PCAP file reader for use during this lab. 

In [1]:
from scapy.all import PcapReader
from scapy.plist import PacketList
from typing import Generator

### Pcap Reader

The code below contains the simple PCAP Reader as provided by ScaPy. It reads from a PCAP file, providing either all packets if the window size is set to 0 or lower, or otherwise only a small window of packets.

In [11]:
def pcap_reader(fpath: str, window_size: int = 50) -> Generator[PacketList, None, None]:
    """PCAP reader generator function.

    Parameters:
    -----------    
    fpath (str): path to pcap file
    window_size (int): number of packets to read at a time

    Output:
    -------
    If windows_size > 0:
        Generator producing PacketLists
    Else:
        PacketList
    """
    reader = PcapReader(fpath)

    if window_size > 0:
        packets = True
        while packets:
            packets = reader.read_all(count=window_size)
            yield packets
    else:
        yield reader.read_all()

The following code gives an example for how to use the PcapReader, as well as what an individual packet looks like.

In [12]:
# The following code reads the entire PCAP file, and outputs all source addresses of TCP packets:
addresses = set()
for packets in pcap_reader('/Users/markvanhezik/Documents/NetworkAndSecurity/lab6/PCAPs/PCAPs_1/easy.pcap', window_size=-1):
    for packet in packets:
        if 'TCP' in packet:
            addresses.add(packet['IP'].src)
    print(f'Source addresses: {addresses}')

Source addresses: {'10.0.0.2', '10.128.0.2'}


In [13]:
# The following code reads a window of 50 PCAP packets at a time, and outputs all destination addresses of TCP packets:
addresses = set()
for packets in pcap_reader('/Users/markvanhezik/Documents/NetworkAndSecurity/lab6/PCAPs/PCAPs_1/easy.pcap', window_size=50):
    for packet in packets:
        if 'TCP' in packet:
            addresses.add(packet['IP'].dst)
print(f'Destination addresses: {addresses}')

Destination addresses: {'10.0.0.2', '10.128.0.2'}


In [14]:
# Finally, this code shows what the final packet of the easy.pcap file looks like:
print(packet.show())

###[ Ethernet ]### 
  dst       = 42:01:0a:f0:00:17
  src       = 42:01:0a:f0:00:01
  type      = IPv4
###[ IP ]### 
     version   = 4
     ihl       = 5
     tos       = 0x0
     len       = 44
     id        = 0
     flags     = DF
     frag      = 0
     ttl       = 55
     proto     = tcp
     chksum    = 0x2f49
     src       = 10.128.0.2
     dst       = 10.0.0.2
     \options   \
###[ TCP ]### 
        sport     = http
        dport     = 4874
        seq       = 361191667
        ack       = 1514175688
        dataofs   = 6
        reserved  = 0
        flags     = SA
        window    = 29200
        chksum    = 0xb8a5
        urgptr    = 0
        options   = [('MSS', 1460)]

None


## Assignment 1

In this assignment, you will analyze entire PCAP files containing different kinds of DDoS attacks. For each attack, analyze the corresponding PCAP file to create a fingerprint of the attack. **You may analyze the PCAP file in full, thus be sure to set the window size to -1.** The fingerprint should contain the type of attack and the victim's IP address. Additionally the source(s) of the attack should be included, print only up to the first 50. Lastly you should include the percentage of traffic that is part of the attack and what percentage of IPs are contributing to the attack. In your malicious traffic percentage calculation only consider traffic **to** the victim IP. For your percentage IPs calculation the Victim's IP should be excluded. **Create a function to generate this fingerprint based on the attack**, containing the following information:  
- Attack type: SYN-FLOOD/DNS-AMP/FRAG  
- Victim IP: X.X.X.X 
- Sources: [IP, IP, ...]  
- Percentage traffic: PERCENTAGE
- Percentage IPs: PERCENTAGE

All PCAP files for this assignment can be found in the folder `PCAPs_1`.  
**In addition to writing the code to detect the different kinds of attacks, we also want you to explain your method of detecting the attack in the markdown cell below the code.**

### Task 1.1 - Detecting Syn Flood Attacks (4 points)

The first PCAP file (easy.pcap) we supply is simple, and will contain a straightforward SYN flood attack. In this attack an attacker keeps sending SYN packages, but never replies to the returned SYN ACK with another ACK. This leaves connections in a sort of limbo. Think about what the unique characteristics and patterns are of such a SYN flood attack, and write code to detect this.

For more information on SYN flood attacks, see chapter 3, page 254 of the book.

In [81]:
from scapy.all import PcapReader
from scapy.plist import PacketList
from typing import Generator

class Syn_Flood:
    def __init__(self):
        self.attacker_info = set()
        self.attack_packet_count = 0 #Amount of packets that are considered malicious
        self.received_packets = dict() #Received packets corresponding to the destination address
        self.all_ips = dict() #All IPs that sent to the destination address
        self.first_attack_nr = dict()
        self.last_attack_nr = (0,0)
        self.wrshark_pack_cnt = 0
        self.fa_src_ip = 0


    def analyze_packet(self, packet):
        try:
            self.wrshark_pack_cnt += 1
            source_address = packet["IP"].src
            destination_address = packet["IP"].dst

            if destination_address not in self.all_ips:
                self.all_ips[destination_address] = set()
                self.all_ips[destination_address].add(source_address)
            else:
                self.all_ips[destination_address].add(source_address)

            tcp_flag = str(packet["TCP"].flags)

            if destination_address not in self.received_packets:
                self.received_packets[destination_address] = 1
            else:
                self.received_packets[destination_address] = self.received_packets[destination_address] + 1

            if ("S" == tcp_flag):
                self.attacker_info.add((source_address, tcp_flag, destination_address))
                self.attack_packet_count += 1
                self.last_attack_nr_temp = self.last_attack_nr
                self.last_attack_nr = (source_address, self.wrshark_pack_cnt)
                if not self.first_attack_nr:
                    self.first_attack_nr[source_address] = self.wrshark_pack_cnt
                    self.fa_src_ip = source_address
            elif("A" == tcp_flag):
                if (source_address, "S", destination_address) in self.attacker_info:
                    self.attack_packet_count -= 1
                    self.attacker_info.remove((source_address, "S", destination_address))
                    if source_address in self.first_attack_nr:
                        self.first_attack_nr.pop(source_address)
                    if(source_address in self.last_attack_nr):
                        self.last_attack_nr = self.last_attack_nr_temp
            else:
                pass
        except:
            pass


    def fingerprint(self):
        sources = []
        victim_ip = ""
        count = 0
        for x in self.attacker_info:
            victim_ip = x[2]
            break
        if len(self.attacker_info) > 0:
            print("Attack type: SYN-FLOOD")
            print(f"Victim IP: {victim_ip}")
            for x in self.attacker_info:
                sources.append(x[0])
                count += 1
                if count == 50:
                    break
            print(f"Sources: {sources}")
            print(f"Percentage traffic: {self.attack_packet_count/self.received_packets[victim_ip] * 100}")
            print(f"Percentage IPs: {len(self.attacker_info) / len(self.all_ips[victim_ip]) * 100}")


def read_pcap(fpath: str, window_size: int = 50) -> Generator[PacketList, None, None]:
    reader = PcapReader(fpath)

    if window_size > 0:
        packets = True
        while packets:
            packets = reader.read_all(count=window_size)
            yield packets
    else:
        yield reader.read_all()


if __name__ == "__main__":
    path = "PCAPs/PCAPs_1/easy.pcap"
    syn_Flood = Syn_Flood()
    for packets in read_pcap(path, -1):
        for packet in packets:
            syn_Flood.analyze_packet(packet)
    syn_Flood.fingerprint()



Attack type: SYN-FLOOD
Victim IP: 10.128.0.2
Sources: ['10.0.0.2']
Percentage traffic: 100.0
Percentage IPs: 100.0


###
All packets containing a SYN flag are added to the set as a tuple. The tuple contains the source ip, flag and destination address. If a successing packet contains an acknowledgement Flag with the same source address and destination address, then it will be removed from the set, otherwise it stays. This means that senders (with the corresponding source ip) won't be removed from the set unless they have send an acknowledgement. This is how the syn flood detection is implementend. When the attacker_info set is empty, we can conclude that no SYN flood attack has occured. If the opposite is the case, then we can print out our results and statistic to STD:OUT.

### Task 1.2 - Detecting DNS Amplification Attacks (4 points)

The second PCAP file (medium.pcap) contains a DNS based amplification attack. This is the more advanced part of this assignment, so be creative in what makes a DNS amplification attack unique, and how you could detect this. Then, write code to detect this attack.

For more information on DNS amplification attacks, see here: https://www.cloudflare.com/learning/ddos/dns-amplification-ddos-attack/

In [82]:
from scapy.all import PcapReader
from scapy.plist import PacketList
from typing import Generator


class DNS_Amp:
    def __init__(self):
        self.attacker_set = set()
        self.victim_set = set()
        self.all_ips = dict()
        self.host_dns_query = set()
        self.received_packets = dict()
        self.attack_packet_count = 0


    def analyze_packet(self, packet):
        try:
            source_address = packet["IP"].src
            destination_address = packet["IP"].dst

            if destination_address not in self.all_ips:
                self.all_ips[destination_address] = set()
                self.all_ips[destination_address].add(source_address)
            else:
                self.all_ips[destination_address].add(source_address)

            if destination_address not in self.received_packets:
                self.received_packets[destination_address] = 1
            else:
                self.received_packets[destination_address] = self.received_packets[destination_address] + 1

            source_port = packet["UDP"].sport
            destination_port = packet["UDP"].dport

            if source_port == 53:
                if((destination_address, source_address, destination_port, source_port) not in self.host_dns_query):
                    self.attacker_set.add(source_address)
                    self.victim_set.add(destination_address)
                    self.attack_packet_count = self.attack_packet_count + 1
            elif destination_port == 53:
                self.host_dns_query.add((source_address, destination_address, source_port, destination_port))
        except IndexError:
            pass
    
    def fingerprint(self):
        sources = []
        count = 0
        for x in self.victim_set:
            victim_ip = x
            break
        if len(self.attacker_set) > 0:
            print("Attack type: DNS-AMP")
            print(f"Victim IP: {victim_ip}")
            for x in self.attacker_set:
                sources.append(x)
                count += 1
                if count == 50:
                    break
            print(f"Sources: {sources}")
            print(f"Percentage traffic: {self.attack_packet_count/self.received_packets[victim_ip] * 100}")
            print(f"Percentage IPs: {len(self.attacker_set) / len(self.all_ips[victim_ip]) * 100}")


def read_pcap(fpath: str, window_size: int = 50) -> Generator[PacketList, None, None]:
    reader = PcapReader(fpath)

    if window_size > 0:
        packets = True
        while packets:
            packets = reader.read_all(count=window_size)
            yield packets
    else:
        yield reader.read_all()


if __name__ == "__main__":
    path = "PCAPs/PCAPs_1/medium.pcap"
    dns_Amp = DNS_Amp()
    for packets in read_pcap(path, -1):
        for packet in packets:
            dns_Amp.analyze_packet(packet)
    dns_Amp.fingerprint()



Attack type: DNS-AMP
Victim IP: 227.213.154.241
Sources: ['95.67.141.109', '95.69.183.183', '253.88.61.187', '99.215.184.207', '123.233.202.237', '101.223.54.219', '245.252.100.23', '115.117.123.125', '253.240.204.45', '251.204.30.187', '253.80.239.221', '117.245.43.85', '99.99.217.121', '251.210.199.145', '113.221.182.179', '123.107.229.113', '113.233.91.27', '93.75.93.83', '253.80.89.79', '95.123.75.157', '99.127.105.87', '111.197.174.145', '251.222.63.179', '95.197.43.17', '105.225.251.87', '253.196.135.151', '121.237.58.117', '253.68.237.15', '241.117.5.125', '101.237.7.237', '95.227.142.47', '255.230.9.247', '95.69.185.149', '109.233.69.113', '113.221.230.143', '123.107.111.111', '105.239.69.251', '253.88.93.209', '95.69.181.151', '253.88.189.251', '253.68.199.87', '95.69.205.189', '117.245.109.145', '251.200.204.177', '99.99.189.59', '97.229.116.125', '101.227.127.45', '97.229.116.77', '109.229.148.117', '251.216.55.125']
Percentage traffic: 80.93992957500959
Percentage IPs: 90.8

###
To indentify the amplification attack we look into the packets to check and see if the DNS response is a response to a query the destination IP address has made. If that's not the case, then we see the DNS response as malicious and remember the IP address of the source. 

### Task 1.3 - Detecting Packet Fragmentation Attacks (4 points)

Another attack vector is exploiting fragmentation. The last PCAP file (hard.pcap) contains an attack that exploits fragmentation of packets. Think about how fragmentation can be used to launch a DDoS attack, and how you can detect it in code. Write that code.

For more information on packet fragmentation attacks, see here: https://www.imperva.com/learn/ddos/ip-fragmentation-attack-teardrop/

In [83]:
from scapy.all import PcapReader
from scapy.plist import PacketList
from typing import Generator


class DNS_Frag:
    def __init__(self):
        self.all_ips = dict()
        self.received_packets = dict()
        self.attack_packet_count = 0
        self.sources = set()
        self.victims = set()
        self.packet_dict = dict()
        self.packet_counter = 0

    def analyze_packet(self, packet):
        self.packet_counter += 1
        try:
            source_address = packet['IP'].src
            destination_address = packet['IP'].dst
            if destination_address not in self.all_ips:
                self.all_ips[destination_address] = set()
            self.all_ips[destination_address].add(source_address)

            if destination_address not in self.received_packets:
                self.received_packets[destination_address] = 1
            else:
                self.received_packets[destination_address] += 1

            if 'IP' in packet and not 'DNS' in packet:
                if (packet['IP'].id, packet['IP'].src, packet['IP'].dst) not in self.packet_dict:
                    self.packet_dict[(packet['IP'].id, packet['IP'].src, packet['IP'].dst)] = []
                self.packet_dict[(packet['IP'].id, packet['IP'].src, packet['IP'].dst)].append((packet['IP'].frag, packet['IP'].flags, packet['IP'].len, self.packet_counter))
        except IndexError:
            pass


    def fingerprint(self):
        b = 8
        packet_header = 20
        sources = set()
        victims = set()
        malicious_traffic_count = 0
        attack_start_set = False
        attack_start = 0
        attack_end = 0
        for packet_id, packet_fragments in self.packet_dict.items():
            fragment_missing = False
            for index, packet_fragment in enumerate(packet_fragments):
                if packet_fragment[2] > 1486:
                    attack_end = packet_fragment[3]
                    fragment_missing = True
                    if not attack_start_set:
                        attack_start = packet_fragment[3]
                        attack_start_set = True
                    break
                if index == 0 and packet_fragment[0] != 0:
                    attack_end = packet_fragment[3]
                    fragment_missing = True
                    if not attack_start_set:
                        attack_start = packet_fragment[3]
                        attack_start_set = True
                    break
                if index > 0 and packet_fragment[0] * b > packet_fragments[index - 1][0] * b + packet_fragments[index - 1][2] - packet_header:
                    attack_end = packet_fragment[3]
                    fragment_missing = True
                    if not attack_start_set:
                        attack_start = packet_fragment[3]
                        attack_start_set = True
                    break
                if packet_fragment == packet_fragments[-1] and 'MF' == packet_fragment[1]:
                    attack_end = packet_fragment[3]
                    fragment_missing = True
                    if not attack_start_set:
                        attack_start = packet_fragment[3]
                        attack_start_set = True
                    break
            if fragment_missing:
                sources.add(packet_id[1])
                victims.add(packet_id[2])
                malicious_traffic_count += len(packet_fragments)

        if sources:
            count = 0
            sources_list = []
            victims_ip = 0
            print("Attack type: FRAG")
            for x in victims:
                victims_ip = x
                break
            for x in sources:
                sources_list.append(x)
                count += 1
                if count == 50:
                    break
            print(f"Victim IP: {victims_ip}")
            print(f"Sources: {sources_list}")
            print(f"Percentage traffic: {100 * malicious_traffic_count / self.received_packets[list(victims)[0]]}")
            print(f"Percentage ip: {100 * len(sources) / len(self.all_ips[list(victims)[0]])}")


def read_pcap(fpath: str, window_size: int = 50) -> Generator[PacketList, None, None]:
    reader = PcapReader(fpath)

    if window_size > 0:
        packets = True
        while packets:
            packets = reader.read_all(count=window_size)
            yield packets
    else:
        yield reader.read_all()


if __name__ == "__main__":
    path = "PCAPs/PCAPs_1/hard.pcap"
    dns_Frag = DNS_Frag()
    for packets in read_pcap(path, -1):
        for packet in packets:
            dns_Frag.analyze_packet(packet)
    dns_Frag.fingerprint()



Attack type: FRAG
Victim IP: 227.213.154.243
Sources: ['47.241.72.109', '117.79.121.239', '253.192.27.23', '109.205.104.187', '97.225.232.55', '95.113.253.159', '251.204.30.187', '245.236.187.53', '119.81.167.45', '119.113.91.111', '107.73.153.247', '99.255.72.109', '117.221.124.253', '99.221.201.245', '109.193.40.245', '117.243.136.155', '95.69.185.173', '55.253.104.85', '119.77.27.239', '107.219.180.185', '117.77.21.115', '245.212.253.124', '111.241.43.119', '109.95.219.23', '97.121.107.147', '251.234.100.205', '251.122.5.109', '101.253.4.61', '111.207.122.117', '251.192.254.19', '117.75.201.117', '117.73.91.181', '117.69.169.121', '117.81.221.115', '179.249.37.117', '109.109.75.21', '95.113.253.215', '119.105.7.251', '105.239.71.109', '103.215.30.93', '45.109.133.115', '253.200.171.83', '161.111.75.87', '63.235.137.29', '117.119.123.57', '103.211.202.51', '123.71.91.187', '57.69.21.243', '109.109.233.55', '115.239.202.213']
Percentage traffic: 95.11636779855017
Percentage ip: 85.470

### 
    The idea is that we analyze each packet that is being fragmented.
    If it's missing a fragment whether it's the initial one (offset 0)
    or the middle one the $n$th fragment's offset is greater than the
    $(n-1)$th offset + $(n-1)$th length. Another case is if the last
    fragment says there should be more fragments in that packet, but
    they were never received by the other side. We can also introduce
    a criterion based on the size of the packet fragment. As stated in
    the link provided to the task, most network use an MTU of 1500bytes.
    Hence, we will also identify the fragments above 1500 as attacks.

## Assignment 2

For this next part of the assignment, you will analyze three larger PCAP files. These PCAP files contain normal traffic, in addition to a possible attack of one of the types from Assignment 1. For this assignment, you will once again need to analyze the files, but you may not look at the file in its entirety. Instead, you may only use a **window size of 50**, to simulate analyzing real time traffic. For each of the files, create a fingerprint where possible as you did in Assignment 1, **but also note when the attack starts, and when it stops**. The format will thus be as follows:  
- Attack type: SYN-FLOOD/DNS-AMP/FRAG  
- Victim IP: X.X.X.X 
- Sources: [IP, IP, ...]  
- Percentage traffic: PERCENTAGE
- Percentage IPs: PERCENTAGE
- Start attack: PACKET_NR
- End attack: PACKET_NR

All PCAP files for this assignment can be found in the folder `PCAPs_2`.   
**Once again, in addition to writing the code to detect the different kinds of attacks, we also want you to explain your method of detecting the attack in the markdown cell below the code.**

### Task 2.1 Large_1 (2 points)

Analyze the file `PCAPs_2/Large_1`.

In [85]:
from struct import pack
from scapy.all import PcapReader
from scapy.plist import PacketList
from typing import Generator

class Syn_Flood:
    def __init__(self):
        self.attacker_info = set()
        self.attack_packet_count = 0 #Amount of packets that are considered malicious
        self.received_packets = dict() #Received packets corresponding to the destination address
        self.all_ips = dict() #All IPs that sent to the destination address
        self.first_attack_nr = dict()
        self.last_attack_nr = (0,0)
        self.wrshark_pack_cnt = 0
        self.fa_src_ip = 0


    def analyze_packet(self, packet):
        try:
            self.wrshark_pack_cnt += 1
            source_address = packet["IP"].src
            destination_address = packet["IP"].dst

            if destination_address not in self.all_ips:
                self.all_ips[destination_address] = set()
                self.all_ips[destination_address].add(source_address)
            else:
                self.all_ips[destination_address].add(source_address)

            tcp_flag = str(packet["TCP"].flags)

            if destination_address not in self.received_packets:
                self.received_packets[destination_address] = 1
            else:
                self.received_packets[destination_address] = self.received_packets[destination_address] + 1

            if ("S" == tcp_flag):
                self.attacker_info.add((source_address, tcp_flag, destination_address))
                self.attack_packet_count += 1
                self.last_attack_nr_temp = self.last_attack_nr
                self.last_attack_nr = (source_address, self.wrshark_pack_cnt)
                if not self.first_attack_nr:
                    self.first_attack_nr[source_address] = self.wrshark_pack_cnt
                    self.fa_src_ip = source_address
            elif("A" == tcp_flag):
                if (source_address, "S", destination_address) in self.attacker_info:
                    self.attack_packet_count -= 1
                    self.attacker_info.remove((source_address, "S", destination_address))
                    if source_address in self.first_attack_nr:
                        self.first_attack_nr.pop(source_address)
                    if(source_address in self.last_attack_nr):
                        self.last_attack_nr = self.last_attack_nr_temp
            else:
                pass
        except:
            pass


    def fingerprint(self):
        sources = []
        victim_ip = ""
        count = 0
        for x in self.attacker_info:
            victim_ip = x[2]
            break
        if len(self.attacker_info) > 0:
            print("Attack type: SYN-FLOOD")
            print(f"Victim IP: {victim_ip}")
            for x in self.attacker_info:
                sources.append(x[0])
                count += 1
                if count == 50:
                    break
            print(f"Sources: {sources}")
            print(f"Percentage traffic: {self.attack_packet_count/self.received_packets[victim_ip] * 100}")
            print(f"Percentage IPs: {len(self.attacker_info) / len(self.all_ips[victim_ip]) * 100}")
            print(f"Start attack: {self.first_attack_nr[self.fa_src_ip]}")
            print(f"End attack: {self.last_attack_nr[1]}")


class DNS_Amp:
    def __init__(self):
        self.attacker_set = set()
        self.victim_set = set()
        self.all_ips = dict()
        self.host_dns_query = set()
        self.received_packets = dict()
        self.attack_packet_count = 0
        self.wrshark_counter, self.first_attack_packet, self.last_attack_packet = 0, 0, 0


    def analyze_packet(self, packet):
        try: 
            source_address = packet["IP"].src
            destination_address = packet["IP"].dst

            if destination_address not in self.all_ips:
                self.all_ips[destination_address] = set()
                self.all_ips[destination_address].add(source_address)
            else:
                self.all_ips[destination_address].add(source_address)

            if destination_address not in self.received_packets:
                self.received_packets[destination_address] = 1
            else:
                self.received_packets[destination_address] = self.received_packets[destination_address] + 1

            source_port = packet["UDP"].sport
            destination_port = packet["UDP"].dport

            if source_port == 53:
                if((destination_address, source_address, destination_port, source_port) not in self.host_dns_query):
                    self.attacker_set.add(source_address)
                    self.victim_set.add(destination_address)
                    self.attack_packet_count = self.attack_packet_count + 1
                    if self.first_attack_packet <= 0:
                        self.first_attack_packet = self.wrshark_counter
                    self.last_attack_packet = self.wrshark_counter
            elif destination_port == 53:
                self.host_dns_query.add((source_address, destination_address, source_port, destination_port))
        except IndexError:
            pass
    
    def fingerprint(self):
        sources = []
        victim_ip = ""
        count = 0
        for x in self.attacker_set:
            victim_ip = x[2]
            break
        if len(self.attacker_set) > 0:
            print("Attack type: DNS-AMP")
            print(f"Victim IP: {victim_ip}")
            for x in self.attacker_set:
                sources.append(x[0])
                count += 1
                if count == 50:
                    break
            print(f"Sources: {sources}")
            print(f"Percentage traffic: {self.attack_packet_count/self.received_packets[victim_ip] * 100}")
            print(f"Percentage IPs: {len(self.attacker_set) / len(self.all_ips[victim_ip]) * 100}")
            print(f"Start attack: {self.first_attack_packet}")
            print(f"End attack: {self.last_attack_packet}")


class DNS_Frag:
    def __init__(self):
        self.all_ips = dict()
        self.received_packets = dict()
        self.attack_packet_count = 0
        self.sources = set()
        self.victims = set()
        self.packet_dict = dict()
        self.packet_counter = 0

    def analyze_packet(self, packet):
        self.packet_counter += 1
        try:
            source_address = packet['IP'].src
            destination_address = packet['IP'].dst
            if destination_address not in self.all_ips:
                self.all_ips[destination_address] = set()
            self.all_ips[destination_address].add(source_address)

            if destination_address not in self.received_packets:
                self.received_packets[destination_address] = 1
            else:
                self.received_packets[destination_address] += 1

            if 'IP' in packet and not 'DNS' in packet:
                if (packet['IP'].id, packet['IP'].src, packet['IP'].dst) not in self.packet_dict:
                    self.packet_dict[(packet['IP'].id, packet['IP'].src, packet['IP'].dst)] = []
                self.packet_dict[(packet['IP'].id, packet['IP'].src, packet['IP'].dst)].append((packet['IP'].frag, packet['IP'].flags, packet['IP'].len, self.packet_counter))
        except IndexError:
            pass


    def fingerprint(self):
        b = 8
        packet_header = 20
        sources = set()
        victims = set()
        malicious_traffic_count = 0
        attack_start_set = False
        attack_start = 0
        attack_end = 0
        for packet_id, packet_fragments in self.packet_dict.items():
            fragment_missing = False
            for index, packet_fragment in enumerate(packet_fragments):
                if packet_fragment[2] > 1486:
                    attack_end = packet_fragment[3]
                    fragment_missing = True
                    if not attack_start_set:
                        attack_start = packet_fragment[3]
                        attack_start_set = True
                    break
                if index == 0 and packet_fragment[0] != 0:
                    attack_end = packet_fragment[3]
                    fragment_missing = True
                    if not attack_start_set:
                        attack_start = packet_fragment[3]
                        attack_start_set = True
                    break
                if index > 0 and packet_fragment[0] * b > packet_fragments[index - 1][0] * b + packet_fragments[index - 1][2] - packet_header:
                    attack_end = packet_fragment[3]
                    fragment_missing = True
                    if not attack_start_set:
                        attack_start = packet_fragment[3]
                        attack_start_set = True
                    break
                if packet_fragment == packet_fragments[-1] and 'MF' == packet_fragment[1]:
                    attack_end = packet_fragment[3]
                    fragment_missing = True
                    if not attack_start_set:
                        attack_start = packet_fragment[3]
                        attack_start_set = True
                    break
            if fragment_missing:
                sources.add(packet_id[1])
                victims.add(packet_id[2])
                malicious_traffic_count += len(packet_fragments)

        if sources:
            count = 0
            sources_list = []
            print("Attack type: FRAG")
            print("Victim IP: ")
            for x in victims:
                print(x)
                break
            print("Sources: ")
            for x in sources:
                sources_list.append(x)
                count += 1
                if count == 50:
                    break
            print(sources_list)
            print(f"Percentage traffic: {100 * malicious_traffic_count / self.received_packets[list(victims)[0]]}")
            print(f"Percentage ip: {100 * len(sources) / len(self.all_ips[list(victims)[0]])}")
            print(f"Attack start: {attack_start}")
            print(f"Attack end: {attack_end}")



def read_pcap(fpath: str, window_size: int = 50) -> Generator[PacketList, None, None]:
    reader = PcapReader(fpath)

    if window_size > 0:
        packets = True
        while packets:
            packets = reader.read_all(count=window_size)
            yield packets
    else:
        yield reader.read_all()


if __name__ == "__main__":
    path = "PCAPs/PCAPs_2/Large_1.pcap"
    syn_Flood = Syn_Flood()
    dns_Amp = DNS_Amp()
    dns_Frag = DNS_Frag()
    for packets in read_pcap(path, 50):
        for packet in packets:
            syn_Flood.analyze_packet(packet)
            dns_Amp.analyze_packet(packet)
            dns_Frag.analyze_packet(packet)
    syn_Flood.fingerprint()
    dns_Amp.fingerprint()
    dns_Frag.fingerprint()



Attack type: SYN-FLOOD
Victim IP: 172.27.224.250
Sources: ['220.65.190.231', '185.208.156.243', '161.90.231.186', '75.219.93.217', '245.153.94.94', '106.57.193.64', '186.7.39.184', '101.12.238.208', '57.108.87.200', '116.37.218.197', '247.9.182.9', '69.157.148.122', '39.162.119.242', '116.47.20.249', '200.43.83.77', '13.34.134.249', '239.99.73.127', '22.59.174.129', '244.42.2.190', '125.198.25.211', '42.215.92.204', '177.227.91.190', '179.166.166.203', '206.127.250.31', '234.120.137.14', '98.233.33.218', '232.233.1.223', '36.199.224.150', '110.124.168.102', '141.225.22.157', '70.8.225.202', '142.54.121.15', '225.230.101.246', '112.233.147.93', '2.233.49.30', '11.202.88.56', '221.232.124.218', '66.100.72.6', '56.119.80.76', '37.108.181.240', '152.121.92.208', '195.249.250.208', '49.172.27.35', '91.88.201.214', '112.156.16.186', '9.177.104.216', '14.51.144.82', '20.177.111.151', '69.42.179.72', '73.84.225.116']
Percentage traffic: 91.6897859451755
Percentage IPs: 99.99944406455522
Start 

###
We've merged the Flood, Amplification and Fragementation detection code. If one of the attacks is in the file, then the detection methods we've written for these attacks should show the output with the additional start and end packet.

### Task 2.2 Large_2 (2 points)

Analyze the file `PCAPs_2/Large_2`.

In [86]:
from struct import pack
from scapy.all import PcapReader
from scapy.plist import PacketList
from typing import Generator

class Syn_Flood:
    def __init__(self):
        self.attacker_info = set()
        self.attack_packet_count = 0 #Amount of packets that are considered malicious
        self.received_packets = dict() #Received packets corresponding to the destination address
        self.all_ips = dict() #All IPs that sent to the destination address
        self.first_attack_nr = dict()
        self.last_attack_nr = (0,0)
        self.wrshark_pack_cnt = 0
        self.fa_src_ip = 0


    def analyze_packet(self, packet):
        try:
            self.wrshark_pack_cnt += 1
            source_address = packet["IP"].src
            destination_address = packet["IP"].dst

            if destination_address not in self.all_ips:
                self.all_ips[destination_address] = set()
                self.all_ips[destination_address].add(source_address)
            else:
                self.all_ips[destination_address].add(source_address)

            tcp_flag = str(packet["TCP"].flags)

            if destination_address not in self.received_packets:
                self.received_packets[destination_address] = 1
            else:
                self.received_packets[destination_address] = self.received_packets[destination_address] + 1

            if ("S" == tcp_flag):
                self.attacker_info.add((source_address, tcp_flag, destination_address))
                self.attack_packet_count += 1
                self.last_attack_nr_temp = self.last_attack_nr
                self.last_attack_nr = (source_address, self.wrshark_pack_cnt)
                if not self.first_attack_nr:
                    self.first_attack_nr[source_address] = self.wrshark_pack_cnt
                    self.fa_src_ip = source_address
            elif("A" == tcp_flag):
                if (source_address, "S", destination_address) in self.attacker_info:
                    self.attack_packet_count -= 1
                    self.attacker_info.remove((source_address, "S", destination_address))
                    if source_address in self.first_attack_nr:
                        self.first_attack_nr.pop(source_address)
                    if(source_address in self.last_attack_nr):
                        self.last_attack_nr = self.last_attack_nr_temp
            else:
                pass
        except:
            pass


    def fingerprint(self):
        sources = []
        victim_ip = ""
        count = 0
        for x in self.attacker_info:
            victim_ip = x[2]
            break
        if len(self.attacker_info) > 0:
            print("Attack type: SYN-FLOOD")
            print(f"Victim IP: {victim_ip}")
            for x in self.attacker_info:
                sources.append(x[0])
                count += 1
                if count == 50:
                    break
            print(f"Sources: {sources}")
            print(f"Percentage traffic: {self.attack_packet_count/self.received_packets[victim_ip] * 100}")
            print(f"Percentage IPs: {len(self.attacker_info) / len(self.all_ips[victim_ip]) * 100}")
            print(f"Start attack: {self.first_attack_nr[self.fa_src_ip]}")
            print(f"End attack: {self.last_attack_nr[1]}")


class DNS_Amp:
    def __init__(self):
        self.attacker_set = set()
        self.victim_set = set()
        self.all_ips = dict()
        self.host_dns_query = set()
        self.received_packets = dict()
        self.attack_packet_count = 0
        self.wrshark_counter, self.first_attack_packet, self.last_attack_packet = 0, 0, 0


    def analyze_packet(self, packet):
        try: 
            source_address = packet["IP"].src
            destination_address = packet["IP"].dst

            if destination_address not in self.all_ips:
                self.all_ips[destination_address] = set()
                self.all_ips[destination_address].add(source_address)
            else:
                self.all_ips[destination_address].add(source_address)

            if destination_address not in self.received_packets:
                self.received_packets[destination_address] = 1
            else:
                self.received_packets[destination_address] = self.received_packets[destination_address] + 1

            source_port = packet["UDP"].sport
            destination_port = packet["UDP"].dport

            if source_port == 53:
                if((destination_address, source_address, destination_port, source_port) not in self.host_dns_query):
                    self.attacker_set.add(source_address)
                    self.victim_set.add(destination_address)
                    self.attack_packet_count = self.attack_packet_count + 1
                    if self.first_attack_packet <= 0:
                        self.first_attack_packet = self.wrshark_counter
                    self.last_attack_packet = self.wrshark_counter
            elif destination_port == 53:
                self.host_dns_query.add((source_address, destination_address, source_port, destination_port))
        except IndexError:
            pass
    
    def fingerprint(self):
        sources = []
        victim_ip = ""
        count = 0
        for x in self.attacker_set:
            victim_ip = x[2]
            break
        if len(self.attacker_set) > 0:
            print("Attack type: DNS-AMP")
            print(f"Victim IP: {victim_ip}")
            for x in self.attacker_set:
                sources.append(x[0])
                count += 1
                if count == 50:
                    break
            print(f"Sources: {sources}")
            print(f"Percentage traffic: {self.attack_packet_count/self.received_packets[victim_ip] * 100}")
            print(f"Percentage IPs: {len(self.attacker_set) / len(self.all_ips[victim_ip]) * 100}")
            print(f"Start attack: {self.first_attack_packet}")
            print(f"End attack: {self.last_attack_packet}")


class DNS_Frag:
    def __init__(self):
        self.all_ips = dict()
        self.received_packets = dict()
        self.attack_packet_count = 0
        self.sources = set()
        self.victims = set()
        self.packet_dict = dict()
        self.packet_counter = 0

    def analyze_packet(self, packet):
        self.packet_counter += 1
        try:
            source_address = packet['IP'].src
            destination_address = packet['IP'].dst
            if destination_address not in self.all_ips:
                self.all_ips[destination_address] = set()
            self.all_ips[destination_address].add(source_address)

            if destination_address not in self.received_packets:
                self.received_packets[destination_address] = 1
            else:
                self.received_packets[destination_address] += 1

            if 'IP' in packet and not 'DNS' in packet:
                if (packet['IP'].id, packet['IP'].src, packet['IP'].dst) not in self.packet_dict:
                    self.packet_dict[(packet['IP'].id, packet['IP'].src, packet['IP'].dst)] = []
                self.packet_dict[(packet['IP'].id, packet['IP'].src, packet['IP'].dst)].append((packet['IP'].frag, packet['IP'].flags, packet['IP'].len, self.packet_counter))
        except IndexError:
            pass


    def fingerprint(self):
        b = 8
        packet_header = 20
        sources = set()
        victims = set()
        malicious_traffic_count = 0
        attack_start_set = False
        attack_start = 0
        attack_end = 0
        for packet_id, packet_fragments in self.packet_dict.items():
            fragment_missing = False
            for index, packet_fragment in enumerate(packet_fragments):
                if packet_fragment[2] > 1486:
                    attack_end = packet_fragment[3]
                    fragment_missing = True
                    if not attack_start_set:
                        attack_start = packet_fragment[3]
                        attack_start_set = True
                    break
                if index == 0 and packet_fragment[0] != 0:
                    attack_end = packet_fragment[3]
                    fragment_missing = True
                    if not attack_start_set:
                        attack_start = packet_fragment[3]
                        attack_start_set = True
                    break
                if index > 0 and packet_fragment[0] * b > packet_fragments[index - 1][0] * b + packet_fragments[index - 1][2] - packet_header:
                    attack_end = packet_fragment[3]
                    fragment_missing = True
                    if not attack_start_set:
                        attack_start = packet_fragment[3]
                        attack_start_set = True
                    break
                if packet_fragment == packet_fragments[-1] and 'MF' == packet_fragment[1]:
                    attack_end = packet_fragment[3]
                    fragment_missing = True
                    if not attack_start_set:
                        attack_start = packet_fragment[3]
                        attack_start_set = True
                    break
            if fragment_missing:
                sources.add(packet_id[1])
                victims.add(packet_id[2])
                malicious_traffic_count += len(packet_fragments)

        if sources:
            count = 0
            sources_list = []
            print("Attack type: FRAG")
            print("Victim IP: ")
            for x in victims:
                print(x)
                break
            print("Sources: ")
            for x in sources:
                sources_list.append(x)
                count += 1
                if count == 50:
                    break
            print(sources_list)
            print(f"Percentage traffic: {100 * malicious_traffic_count / self.received_packets[list(victims)[0]]}")
            print(f"Percentage ip: {100 * len(sources) / len(self.all_ips[list(victims)[0]])}")
            print(f"Attack start: {attack_start}")
            print(f"Attack end: {attack_end}")



def read_pcap(fpath: str, window_size: int = 50) -> Generator[PacketList, None, None]:
    reader = PcapReader(fpath)

    if window_size > 0:
        packets = True
        while packets:
            packets = reader.read_all(count=window_size)
            yield packets
    else:
        yield reader.read_all()


if __name__ == "__main__":
    path = "PCAPs/PCAPs_2/Large_2.pcap"
    syn_Flood = Syn_Flood()
    dns_Amp = DNS_Amp()
    dns_Frag = DNS_Frag()
    for packets in read_pcap(path, 50):
        for packet in packets:
            syn_Flood.analyze_packet(packet)
            dns_Amp.analyze_packet(packet)
            dns_Frag.analyze_packet(packet)
    syn_Flood.fingerprint()
    dns_Amp.fingerprint()
    dns_Frag.fingerprint()



###
We've merged the Flood, Amplification and Fragementation detection code. If one of the attacks is in the file, then the detection methods we've written for these attacks should show the output with the additional start and end packet. By the looks of it, it doesn't contain any attacks.


### Task 2.1 Large_3 (4 points)

The file `PCAPs_2/Large_3` contains an unidentified attack other than the ones you have seen before in this lab. In this task you should take a look at the PCAP and try to identify this attack. After identifying the type of attack, write code to detect the attack and create a fingerprint. 

**In your explanation also explain the attack in addition to how you detected it.**

In [87]:

class ICMP_flood:
    def __init__(self):
        self.all_ips = dict()
        self.received_packets = dict()
        self.attack_packet_count = 0
        self.sources = set()
        self.victims = set()
        self.packet_dict = dict()
        self.packet_counter = 0
        self.first_one = None
        self.last_one = None

    def analyze_packet(self, packet):
        self.packet_counter += 1
        try:
            source_address = packet['IP'].src
            destination_address = packet['IP'].dst
            if destination_address not in self.all_ips:
                self.all_ips[destination_address] = set()
            self.all_ips[destination_address].add(source_address)

            if destination_address not in self.received_packets:
                self.received_packets[destination_address] = 1
            else:
                self.received_packets[destination_address] += 1

            if 'ICMP' in packet and packet['ICMP'].type == 8:
                self.last_one = self.packet_counter
                if self.first_one is None:
                    self.first_one = self.packet_counter
                if (packet['IP'].src, packet['IP'].dst) not in self.packet_dict:
                    self.packet_dict[(packet['IP'].src, packet['IP'].dst)] = []
                self.packet_dict[(packet['IP'].src, packet['IP'].dst)].append(self.packet_counter)
        except IndexError:
            pass


    def fingerprint(self):
        sources = set()
        victims = set()
        malicious_traffic_count = 0
        for (src, dst), packet_list in self.packet_dict.items():
            sources.add(src)
            victims.add(dst)
            malicious_traffic_count += len(packet_list)

        if sources:
            count = 0
            sources_list = []
            victims_ip = ""
            print("Attack type: ICMP")
            for x in victims:
                victims_ip = x
                break
            for x in sources:
                sources_list.append(x)
                count += 1
                if count == 50:
                    break
            print(f"Victim IP: {victims_ip}")
            print(f"Sources: {sources_list}")
            print(f"Percentage traffic: {100 * malicious_traffic_count / self.received_packets[list(victims)[0]]}")
            print(f"Percentage ip: {100 * len(sources) / len(self.all_ips[list(victims)[0]])}")
            print(f"Attack start: {self.first_one}")
            print(f"Attack end: {self.last_one}")


icmp_flood = ICMP_flood()
for packets in pcap_reader('PCAPs/PCAPs_2/Large_3.pcap', window_size=50):
        for packet in packets:
            icmp_flood.analyze_packet(packet)

icmp_flood.fingerprint()

Attack type: ICMP
Victim IP: 172.27.224.250
Sources: ['39.202.157.255', '72.41.160.51', '97.24.58.65', '108.1.166.69', '168.82.32.12', '181.27.31.108', '229.139.53.211', '190.244.247.158', '142.244.84.116', '181.114.23.122', '11.100.153.232', '97.64.64.42', '236.104.229.172', '14.170.8.38', '249.61.113.52', '84.35.73.78', '246.215.18.25', '9.194.66.113', '10.194.243.85', '207.138.74.165', '190.228.9.10', '35.90.33.87', '92.46.73.237', '32.79.127.32', '3.63.248.181', '99.75.65.4', '88.225.133.114', '200.248.142.48', '227.255.176.117', '40.152.16.78', '28.248.194.20', '218.89.51.115', '250.182.75.164', '11.21.127.78', '54.137.92.239', '115.75.134.73', '32.0.204.7', '128.97.75.177', '81.33.212.97', '164.103.101.208', '90.22.223.232', '45.200.170.116', '254.98.254.211', '162.244.118.250', '158.244.78.61', '228.8.131.62', '238.131.98.86', '130.230.141.230', '244.130.114.179', '219.167.20.123']
Percentage traffic: 53.198563638781756
Percentage ip: 99.99166736105325
Attack start: 2933
Attack 

###
After looking at the trace file we observed that there was an unusual
amount of ICMP echo messages concentrated in some places. We researched
the topic and we learnt that there is a type of DDoS attack based
on ICMP pings. Then, we designed algorithm that is supposed to detect
these ICMP pings and provide us with the fingerprint of the attack.
It detects all ICMP messages that are of type 8 (the ping messages).
From the fingerprint we can learn how big the attack was or another
way to think about it - how likely it is that the detected case
was an attack in the first place.